# CrewAI and Weaviate Getting Started Example

This notebook shows you how industry-specific agents use web search and Weaviate blogs to track trends and uncover AI use cases across biomedical, healthcare, and finance.

## Import Libraries and Dependencies

At the time of creating this notebook, we're using the `weaviate-client` version `4.15.4` and `crewai` version `0.134.0`.

In [ ]:
!pip install --quiet weaviate-client==4.15.4

In [ ]:
!pip install --quiet crewai==0.134.0

In [5]:
import os
from dotenv import load_dotenv

from crewai_tools import WeaviateVectorSearchTool
import weaviate
from weaviate.classes.init import Auth

from crewai import Agent
from crewai_tools import WeaviateVectorSearchTool
from crewai import Task
from crewai import Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool

load_dotenv()


True

## Connect to Serper API and OpenAI

You can get 2,500 of free credits with Serper [here](https://serper.dev/)!

In [9]:
os.environ["SERPER_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

## Connect to Weaviate

Create a free sandbox cluster on [Weaviate Cloud](https://console.weaviate.cloud/)!

In [21]:
WCD_CLUSTER_URL = os.getenv("WCD_CLUSTER_URL")
WCD_CLUSTER_KEY = os.getenv("WCD_CLUSTER_KEY")

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WCD_CLUSTER_URL,
    auth_credentials=Auth.api_key(WCD_CLUSTER_KEY),
    headers={"X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")}
)

print(client.is_ready())

True


## Connect to Existing Weaviate Collection

You'll want to connect to your existing Weaviate collection. If you don't have one already, you can follow this import notebook [here](https://github.com/weaviate/recipes/blob/main/integrations/Weaviate-Import-Example.ipynb)!

In [ ]:
weaviate_collection = client.collections.get("WeaviateBlogChunk")

## Initialize the Tools

### Weaviate Search Tool

In [23]:
weaviate_tool = WeaviateVectorSearchTool(
    collection_name='WeaviateBlogChunk', 
    limit=4,
    weaviate_cluster_url=WCD_CLUSTER_URL,
    weaviate_api_key=WCD_CLUSTER_KEY,
)

### Web Search Tool

In [24]:
search_tool = SerperDevTool()

## Create the Agents

### BioMed Marketing Agent

In [30]:
BiomedicalMarketingAgent = Agent(
    role='Industry researcher focused on biomedical trends and their applications in AI',
    goal='Continuously track the latest biomedical advancements and identify how Weaviate’s features can support AI applications in biomedical research, diagnostics, and personalized medicine.',
    backstory='As a former biomedical product marketer turned AI strategist, you understand the complex language and regulatory landscape of biomedical innovation. With a keen eye on genomics, clinical research, and medical devices, it now leverages LLMs and vector search to explore how Weaviate’s capabilities can streamline scientific discovery and patient-centric campaigns.',
    llm="gpt-4o-mini",
    tools=[search_tool, weaviate_tool],
    verbose=True
)

### Healthcare Marketing Agent 

In [29]:
HealthcareMarketingAgent = Agent(
    role='AI-savvy marketer specializing in healthcare systems, digital health, and patient engagement.',
    goal='Stay updated on healthcare policy shifts, digital health trends, and explore how Weaviate’s features can optimize workflows in hospital systems, EHR integration, and health communication.',
    backstory='Rooted in public health communications, this agent has evolved into a digital health consultant. You focus on how retrieval-augmented generation (RAG), semantic search, and hybrid models can be applied to solve healthcare-specific challenges—from patient triage to clinical support systems.',
    llm="gpt-4o-mini",
    tools=[search_tool, weaviate_tool],
    verbose=True
)

### Financial Marketing Agent

In [28]:
FinancialMarketingAgent = Agent(
    role='Insight analyst exploring innovations in finance, wealth tech, and regulatory tech',
    goal='Monitor financial sector trends including AI in trading, compliance automation, and client advisory, and assess how Weaviate’s tools can enable cutting-edge financial applications.',
    backstory='With experience at a fintech startup and a background in capital markets, this agent specializes in using structured + unstructured data to surface insights for analysts and advisors. Now, it’s looking into how vector databases and LLMs can automate tasks like fraud detection, investor personalization, and market research.',
    llm="gpt-4o-mini",
    tools=[search_tool, weaviate_tool],
    verbose=True
)

## Define the Agents Tasks

### BioMed Agent Task

In [34]:
biomed_agent_task = Task(
    description = """
        Conduct a thorough research about {weaviate_feature}
        Make sure you find any interesting and relevant information using the web and Weaviate blogs.
    """,
    expected_output = """
        Write an industry specific analysis of why this Weaviate feature would be useful for your industry of expertise.
    """,
    agent = BiomedicalMarketingAgent
)

### Healthcare Agent Task

In [35]:
healthcare_agent_task = Task(
    description = """
        Conduct a thorough research about {weaviate_feature}
        Make sure you find any interesting and relevant information using the web and Weaviate blogs.
    """,
    expected_output = """
        Write an industry specific analysis of why this Weaviate feature would be useful for your industry of expertise.
    """,
    agent = HealthcareMarketingAgent
)

### Financial Agent Task

In [36]:
financial_agent_task = Task(
    description = """
        Conduct a thorough research about {weaviate_feature}
        Make sure you find any interesting and relevant information using the web and Weaviate blogs.
    """,
    expected_output = """
        Write an industry specific analysis of why this Weaviate feature would be useful for your industry of expertise.
    """,
    agent = FinancialMarketingAgent
)

### List out the Weaviate Features

In [41]:
weaviate_features = [
  {"weaviate_feature": "MUVERA"},
  {"weaviate_feature": "Multi-tenancy"},
  {"weaviate_feature": "Compliance"},
  {"weaviate_feature": "Hybrid Search"}
]

### Assemble the Crew

In [45]:
blog_crew = Crew(
    agents=[BiomedicalMarketingAgent, HealthcareMarketingAgent, FinancialMarketingAgent],
    tasks=[biomed_agent_task, healthcare_agent_task, financial_agent_task],
    # verbose=True, # uncomment if you'd like to see the full execution from the kickoff
    # planning=True
)

## Query Time

In [46]:
result = blog_crew.kickoff_for_each(inputs=weaviate_features)

print(result)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Industry researcher focused on biomedical trends and their applications in AI                           │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Conduct a thorough research about MUVERA                                                               │
│          Make sure you find any interesting and relevant information using the web and Weaviate blogs.          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Industry researcher focused on biomedical trends and their applications in AI                           │
│                                                                                                                 │
│  Thought: I need to gather information about MUVERA and its relevance to the biomedical industry, particularly  │
│  how it intersects with Weaviate's capabilities. To do this comprehensively, I'll first search the internet     │
│  for relevant information about MUVERA.                                                                         │
│                                                                                                                 │
│  Using Tool: Search the internet                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"MUVERA biomedical applications Weaviate\"}"                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'MUVERA biomedical applications Weaviate', 'type': 'search', 'num': 10, 'engine':   │
│  'google'}, 'organic': [{'title': 'More efficient multi-vector embeddings with MUVERA - Weaviate', 'link':      │
│  'https://weaviate.io/blog/muvera', 'snippet': 'Weaviate 1.31 implements the MUVERA encoding algorithm for      │
│  multi-vector embeddings. In this blog, we dive the algorithm in detail, ...', 'position': 1}, {'title':        │
│  'MUVERA with Rajesh Jayaram and Roberto Esposito - Weaviate ...', 'link':                                      │
│  'https://creators.spotify.com/pod/profile/weaviate/episodes/MUVERA-with-Rajesh-Jayaram-and-Roberto-Esposito--  │
│  -Weaviate-Podcast-123-e33fnpi', 'snippet': 'This podcast dives into all things IReRa, XMC, DSPy compilation,   │
│  and applications in Biomedical NLP and Recommendation! I hope you find this ...', 'position': 2}, {'title':    │
│  'Multi-vector encodings | Weaviate', 'link':                                                                   │
│  'https://weaviate.io/developers/weaviate/configuration/compression/multi-vectors', 'snippet': 'Missing:        │
│  biomedical applications', 'position': 3}, {'title': 'MUVERA with Rajesh Jayaram and Roberto Esposito —         │
│  Weaviate ...', 'link':                                                                                         │
│  'https://connorshorten300.medium.com/muvera-with-rajesh-jayaram-and-roberto-esposito-weaviate-podcast-123-b8c  │
│  07076ac3d', 'snippet': "While MUVERA's primary goal is efficient retrieval, the underlying principles of       │
│  representing sets of vectors can have broader applications. LSH ...", 'position': 4}, {'title': 'Weaviate on   │
│  Snowpark Container Services, new Python client (v4 ...', 'link':                                               │
│  'https://newsletter.weaviate.io/p/weaviate-snowpark-container-services-new-python-client-v4-multimodal-produc  │
│  t-discovery', 'snippet': 'Discover how this system revolutionizes Extreme Multi-Label Classification using     │
│  DSPy compilation, with applications in Biomedical NLP and ...', 'position': 5}, {'title': 'MUVERA: Making      │
│  multi-vector retrieval as fast as single-vector search', 'link':                                               │
│  'https://research.google/blog/muvera-making-multi-vector-retrieval-as-fast-as-single-vector-search/',          │
│  'snippet': 'We intro...                                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Industry researcher focused on biomedical trends and their applications in AI                           │
│                                                                                                                 │
│  Thought: I found several relevant sources about MUVERA and its applications, particularly in the context of    │
│  Weaviate. I will delve deeper into Weaviate's official blog and documentation because they may provide         │
│  specific insights related to biomedical applications.                                                          │
│                                                                                                                 │
│  Using Tool: WeaviateVectorSearchTool                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Insight analyst exploring innovations in finance, wealth tech, and regulatory tech                      │
│                                                                                                                 │
│  Thought: I have gathered some relevant information about Hybrid Search and its applications. Now, I will also  │
│  search the Weaviate blogs to find more specific insights on Hybrid Search within the context of the financial  │
│  industry.                                                                                                      │
│                                                                                                                 │
│  Using Tool: WeaviateVectorSearchTool                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"Hybrid Search in the financial industry\"}"                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "content": "---\ntitle: Hybrid Search Explained\nslug: hybrid-search-explained\nauthors: [erika]\ndate:      │
│  2023-01-03\ntags: ['concepts', 'search']\nimage: ./img/hero.png\ndescription: \"Learn about the new hybrid     │
│  search feature that enables you to combine dense and sparse vectors to deliver the best of both search         │
│  methods.\"\n---\n![Hybrid search](./img/hero.png)\n\n<!-- truncate -->\n\nHybrid search is a technique that    │
│  **combines** multiple search algorithms to improve the accuracy and relevance of search results. It uses the   │
│  best features of both keyword-based search algorithms with vector search techniques. By leveraging the         │
│  strengths of different algorithms, it provides a more effective search experience for users. The [Hybrid       │
│  search](/developers/weaviate/api/graphql/search-operators#hybrid) feature was introduced in Weaviate 1.17. It  │
│  uses sparse and dense vectors to represent the meaning and context of search queries and documents.",          │
│    "author": null                                                                                               │
│  }{                                                                                                             │
│    "content": ":::\n\n## How does hybrid search work, exactly? ![Hybrid search main image - a figurative image  │
│  of Weaviate bot combining results from vector and keyword search to produce hybrid                             │
│  search](./img/hybrid-search-illurated.png)\n\nHere is an example of a hybrid search:\n\n```python\nresponse =  │
│  (\n    client.query\n    .get(\"JeopardyQuestion\", [\"question\", \"answer\"])\n                              │
│  .with_hybrid(query=\"food\", alpha=0.5)\n    .with_limit(5)\n    .do()\n)\n```\n\nAs mentioned, a hybrid       │
│  search is really two searches under-the-hood. It performs a vector search (similar to `nearText` or            │
│  `nearVector` in Weaviate) to find most similar objects to the vector of your query. Meanwhile, it also         │
│  performs a keyword search, which ranks results based on how often the query terms occur. In other words, a     │
│  hybrid search performs both of these searches and combines the results.",                                      │
│    "author": null                                                                                               │
│  }{                                                                                                             │
│    "content": "We have prepared [this short                                                                     │
│  survey](https://docs.google.com/forms/d/e/1FAIpQLSfxekC2eh5Xkl0iJYEAoOkfhK4FA5AO-O...                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Insight analyst exploring innovations in finance, wealth tech, and regulatory tech                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### The Importance of Hybrid Search in the Financial Sector                                                    │
│                                                                                                                 │
│  In today's fast-evolving financial landscape, the ability to efficiently manage and retrieve data is           │
│  paramount for institutions seeking to maintain a competitive edge. Hybrid Search, a feature introduced in      │
│  Weaviate 1.17, enables organizations to harness the power of both keyword-based and vector search algorithms.  │
│  This unique blend of search methodologies is particularly beneficial for the financial industry, where         │
│  accuracy and context are crucial for decision-making.                                                          │
│                                                                                                                 │
│  #### Enhancing Data Retrieval                                                                                  │
│                                                                                                                 │
│  Hybrid Search combines the precision of keyword search—which emphasizes the frequency of terms in a            │
│  dataset—with the semantic understanding provided by vector search, known for its capability to capture         │
│  context and meaning. For financial institutions, this means that queries can return not only relevant          │
│  documents based on strict term matches but also more nuanced results that consider the context of those        │
│  terms. This is essential for regulatory compliance, as automated systems can identify documents related to     │
│  specific regulations while also interpreting the implication of terms like "compliance," "risk," or            │
│  "liquidity" based on broader context.                                                                          │
│                                                                                                                 │
│  #### Improved Fraud Detection                                                                                  │
│                                                                                                                 │
│  In fraud detection processes, Hybrid Search can significantly enhance a financial firm's ability to identify   │
│  suspicious activity. For instance, a query regarding unusual transaction patterns can leverage Hybrid Search   │
│  to return results that include both transactions matching specific keywords, such as "large transfer" or       │
│  "foreign account," as well as those that are semantically similar based on historical data patterns. This      │
│  combination can help financial analysts detect potential fraud that would not be evident through keywords      │
│  alone.                                                                                                         │
│                                                                                                                 │
│  #### Facilitating Investor Personalization                                                                     │
│                                                                                                                 │
│  Another critical application of Hybrid Search is in cr

[CrewOutput(raw="The gathered information provides a comprehensive view of MUVERA's capabilities and its potential applications within the financial sector, particularly in enhancing operations in fintech, compliance automation, and client advisory. Here’s a detailed analysis.\n\n---\n\n**Industry Specific Analysis: MUVERA and its Usefulness in the Financial Sector**\n\nMUVERA, an advanced feature implemented in Weaviate’s 1.31 release, introduces a new approach to managing multi-vector embeddings, delivering significant performance enhancements crucial for the demanding requirements of the financial sector. This analysis delineates how MUVERA can revolutionize various functions within finance, namely fraud detection, investor personalization, and market research.\n\n### 1. Enhanced Performance: Speed and Efficiency\n\nOne of the standout advantages of MUVERA is its ability to compress multi-vector representations into fixed-dimensional encodings. This process results in remarkable mem